Обучаем модель с нелинейной архитектурой по базе HH, задаем кастомную сигнатуру для TensorFlow Serving. 
В этом задании нужно обучить нейронную сеть с нелинейной архитектурой и создать кастомную сигнатуру этой модели для загрузки в TensorFlow Serving.  

Для этого поработаем с базой резюме с HeadHunter и обучим нейронную сеть для решения задачи оценки зарплаты пользователя по указанным данным.  

Весь код по обработке данных уже готов и полностью в вашем распоряжении. Вам же необходимо выполнить следующие этапы:    

1. Запустите все ячейки ноутбука с загрузкой и предобработкой данных.

2. Создайте и обучите модель нейронной сети с тремя входами для обработки: 
 * числовых данных
 * текстовых данных "должность" 
 * текстовых данных "опыт работы"

3. Задайте кастомную сигнатуру модели и сохраните обученную нейронную сеть. Выведите сигнатуру сохраненной модели.

4. Запустите TensorFlow Serving в Colab.

5. Отправьте несколько примеров тестового набора данных на сервер для предсказания при помощи POST-запроса. 

6. Полученные результаты сравните с реальными значениями зарплат.

# Start. Запустите все ячейки ноутбука с загрузкой и предобработкой данных.

## Импорт библиотек

In [4]:
# Модуль для работы с операционной системой
import os

# Работа с массивами данных
import numpy as np 

# Работа с табличными данными
import pandas as pd

# Библиотека tensorflow
import tensorflow as tf

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для создания модели нейронной сети
from tensorflow.keras.models import Model

# Слои для создания нейронной сети
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input, concatenate

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Разделение данных на выборки
from sklearn.model_selection import train_test_split

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt

%matplotlib inline

## Загрузка и подготовка данных

In [5]:
# скачиваем базу
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l10/hh_fixed.csv', None, quiet=True)

# Чтение файла базы данных
df = pd.read_csv('hh_fixed.csv', index_col=0)

# Вывод количества резюме и числа признаков
print(df.shape)

df.head()

(62967, 12)


,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 29 лет , родился 16 мая 1989",40000 руб.,Специалист по поддержке чата(support team) дом...,"Новороссийск , готов к переезду (Анапа, Геленд...",полная занятость,полный день,Опыт работы 3 года 9 месяцев Специалист по по...,"ООО ""Гольфстрим""",Генеральный директор,Высшее образование 2011 Международный юридиче...,26.04.2019 08:04,Не указано
1,"Мужчина , 38 лет , родился 25 мая 1980",40000 руб.,Системный администратор,"Новосибирск , м. Березовая роща , не готов к ...",полная занятость,полный день,Опыт работы 11 лет 11 месяцев Системный админ...,ООО «Завод модульных технологий»,Системный администратор,Высшее образование 2002 Новосибирский государс...,26.04.2019 04:30,Не указано
2,"Мужчина , 35 лет , родился 14 июня 1983",300000 руб.,DevOps TeamLead / DevOps архитектор,"Москва , готов к переезду , готов к редким ком...",полная занятость,полный день,Опыт работы 12 лет 11 месяцев DevOps TeamLead...,Банк ВТБ (ПАО),Начальник отдела методологии разработки (DevOp...,DevOps TeamLead / DevOps архитектор 300 000 ру...,09.04.2019 14:40,Не указано
3,"Мужчина , 33 года , родился 2 августа 1985",180000 руб.,Руководитель IT отдела,"Москва , м. Щукинская , не готов к переезду ,...","частичная занятость, полная занятость","удаленная работа, полный день",Опыт работы 15 лет 10 месяцев Руководитель IT...,"""Ай-Теко"", ведущий российский системный интегр...",Старший системный администратор,Руководитель IT отдела 180 000 руб. Информацио...,09.04.2019 14:39,Имеется собственный автомобиль
4,"Мужчина , 22 года , родился 1 сентября 1996",40000 руб.,Junior Developer,"Москва , м. Юго-Западная , не готов к переезд...","стажировка, частичная занятость, проектная работа","гибкий график, удаленная работа",Опыт работы 1 год 1 месяц Junior Developer 40...,R-Style SoftLab,Менеджер IT-проектов,Junior Developer 40 000 руб. Информационные те...,29.03.2019 12:40,Не указано


In [6]:
# Настройка номеров столбцов

COL_SEX_AGE     = df.columns.get_loc('Пол, возраст')
COL_SALARY      = df.columns.get_loc('ЗП')
COL_POS_SEEK    = df.columns.get_loc('Ищет работу на должность:')
COL_POS_PREV    = df.columns.get_loc('Последеняя/нынешняя должность')
COL_CITY        = df.columns.get_loc('Город')
COL_EMPL        = df.columns.get_loc('Занятость')
COL_SCHED       = df.columns.get_loc('График')
COL_EXP         = df.columns.get_loc('Опыт (двойное нажатие для полной версии)')
COL_EDU         = df.columns.get_loc('Образование и ВУЗ')
COL_UPDATED     = df.columns.get_loc('Обновление резюме')

### Преобразование числовых данных

In [7]:
### Параметрические данные для функций разбора ###

# Курсы валют для зарплат
currency_rate = {'usd'    : 65.,
                 'kzt'    : 0.17,
                 'грн'    : 2.6,
                 'белруб' : 30.5,
                 'eur'    : 70.,
                 'kgs'    : 0.9,
                 'сум'    : 0.007,
                 'azn'    : 37.5
                }

# Списки и словари для разбиения на классы
# Для ускорения работы добавлен счетчик классов, который будет вычислен ниже

# Список порогов возраста
age_class = [0, [18, 23, 28, 33, 38, 43, 48, 53, 58, 63]]

# Список порогов опыта работы в месяцах
experience_class = [0, [7, 13, 25, 37, 61, 97, 121, 157, 193, 241]]

# Классы городов
city_class = [0, 
              {'москва'          : 0,
               'санкт-петербург' : 1,
               'новосибирск'     : 2,
               'екатеринбург'    : 2,
               'нижний новгород' : 2,
               'казань'          : 2,
               'челябинск'       : 2,
               'омск'            : 2,
               'самара'          : 2,
               'ростов-на-дону'  : 2,
               'уфа'             : 2,
               'красноярск'      : 2,
               'пермь'           : 2,
               'воронеж'         : 2,
               'волгоград'       : 2,
               'прочие города'   : 3
              }]

# Классы занятости
employment_class = [0, 
                    {'стажировка'          : 0,
                     'частичная занятость' : 1,
                     'проектная работа'    : 2,
                     'полная занятость'    : 3
                    }]

# Классы графика работы
schedule_class = [0, 
                  {'гибкий график'         : 0,
                   'полный день'           : 1,
                   'сменный график'        : 2,
                   'удаленная работа'      : 3
                  }]

# Классы образования
education_class = [0,
                   {'высшее образование'   : 0,
                    'higher education'     : 0,
                    'среднее специальное'  : 1,
                    'неоконченное высшее'  : 2,
                    'среднее образование'  : 3
                   }]

In [8]:
# Вычисление счетчиков для данных разбиения

for class_desc in [age_class,
                   experience_class,
                   city_class,
                   employment_class,
                   schedule_class,
                   education_class]:
    if isinstance(class_desc[1], list):
        class_desc[0] = len(class_desc[1]) + 1
    else:
        class_desc[0] = max(class_desc[1].values()) + 1

In [9]:
 # Получение one hot encoding представления значения класса
 
 def int_to_ohe(arg, class_list):
  
    # Определение размерности выходного вектора
    num_classes = class_list[0]

    # Поиск верного интервала для входного значения
    for i in range(num_classes - 1):
        if arg < class_list[1][i]:
            cls = i                       # Интервал найден - назначение класса
            break
    else:                                 # Внимание: for/else
        cls = num_classes - 1             # Интервал не найден - последний класс

    # Возврат в виде one hot encoding-вектора
    return utils.to_categorical(cls, num_classes)

In [10]:
# Общая функция преобразования строки к multi-вектору
# На входе данные и словарь сопоставления подстрок классам

def str_to_multi(arg, class_dict):
    # Определение размерности выходного вектора
    num_classes = class_dict[0]

    # Создание нулевого вектора
    result = np.zeros(num_classes)
    
    # Поиск значения в словаре и, если найдено,
    # выставление 1. на нужной позиции
    for value, cls in class_dict[1].items():
        if value in arg:
            result[cls] = 1.

    return result

In [11]:
# Разбор значений пола, возраста

base_update_year = 2019

def extract_sex_age_years(arg):
    # Ожидается, что значение содержит "мужчина" или "женщина"
    # Если "мужчина" - результат 1., иначе 0.
    sex = 1. if 'муж' in arg else 0.

    try:
        # Выделение года и вычисление возраста
        years = base_update_year - int(re.search(r'\d{4}', arg)[0])

    except (IndexError, TypeError, ValueError):
        # В случае ошибки год равен 0
        years = 0

    return sex, years

In [12]:
# Преобразование значения возраста в one hot encoding

def age_years_to_ohe(arg):
    return int_to_ohe(arg, age_class)

In [13]:
# Преобразование данных об опыте работы в one hot encoding

def experience_months_to_ohe(arg):
    return int_to_ohe(arg, experience_class)

In [14]:
# Разбор значения зарплаты

def extract_salary(arg):
    try:
        # Выделение числа и преобразование к float
        value = float(re.search(r'\d+', arg)[0])

        # Поиск символа валюты в строке, и, если найдено,
        # приведение к рублю по курсу валюты
        for currency, rate in currency_rate.items():
            if currency in arg:
                value *= rate
                break

    except TypeError:
        # Если не получилось выделить число - вернуть 0
        value = 0.

    return value / 1000.                  # В тысячах рублей

In [15]:
# Разбор данных о городe и преобразование в one hot encoding

def extract_city_to_ohe(arg):
    # Определение размерности выходного вектора
    num_classes = city_class[0]

    # Разбивка на слова
    split_array = re.split(r'[ ,.:()?!]', arg)
    city = split_array[0]  # Берем первое слово строки - это и есть город
    
    # Поиск города в словаре и присвоение класса
    city_cls = city_class[1].get(city.lower(), -1)
    
    # Если город не в city_class - значит его класс "прочие города"
    if city_cls < 0:   
        city_cls = num_classes - 1

    # Возврат в виде one hot encoding-вектора
    return utils.to_categorical(city_cls, num_classes)

In [16]:
# Разбор данных о желаемой занятости и преобразование в multi

def extract_employment_to_multi(arg):
    return str_to_multi(arg, employment_class)

In [17]:
# Разбор данных о желаемом графике работы и преобразование в multi

def extract_schedule_to_multi(arg):
    return str_to_multi(arg, schedule_class)

In [18]:
# Разбор данных об образовании и преобразование в multi

def extract_education_to_multi(arg):
    result = str_to_multi(arg, education_class)
    
    # Поправка: неоконченное высшее не может быть одновременно с высшим
    if result[2] > 0.:
        result[0] = 0.
    
    return result

In [19]:
# Разбор данных об опыте работы - результат в месяцах

def extract_experience_months(arg):
    try:
        # Выделение количества лет, преобразование в int
        years = int(re.search(r'(\d+)\s+(год.?|лет)', arg)[1])

    except (IndexError, TypeError, ValueError):
        # Неудача - количество лет равно 0
        years = 0
    
    try:
        # Выделение количества месяцев, преобразование в int
        months = int(re.search(r'(\d+)\s+месяц', arg)[1])

    except (IndexError, TypeError, ValueError):
        # Неудача - количество месяцев равно 0
        months = 0

    # Возврат результата в месяцах
    return years * 12 + months

Функции подготовки выборок

In [20]:
def extract_row_data(row):
  
    # Извлечение и преобразование данных
    sex, age = extract_sex_age_years(row[COL_SEX_AGE])      # Пол, возраст
    sex_vec = np.array([sex])                               # Пол в виде вектора
    age_ohe = age_years_to_ohe(age)                         # Возраст в one hot encoding
    city_ohe = extract_city_to_ohe(row[COL_CITY])           # Город
    empl_multi = extract_employment_to_multi(row[COL_EMPL]) # Тип занятости
    sсhed_multi = extract_schedule_to_multi(row[COL_SCHED]) # График работы
    edu_multi = extract_education_to_multi(row[COL_EDU])    # Образование
    exp_months = extract_experience_months(row[COL_EXP])    # Опыт работы в месяцах
    exp_ohe = experience_months_to_ohe(exp_months)          # Опыт работы в one hot encoding
    salary = extract_salary(row[COL_SALARY])                # Зарплата в тысячах рублей
    salary_vec = np.array([salary])                         # Зарплата в виде вектора

    # Объединение всех входных данных в один общий вектор
    x_data = np.hstack([sex_vec,
                        age_ohe, 
                        city_ohe,
                        empl_multi,
                        sсhed_multi,
                        edu_multi,
                        exp_ohe])
    
    # Возврат входных данных и выходных (зарплаты)
    return x_data, salary_vec


# Создание общей выборки
def construct_train_data(row_list):
    x_data = []
    y_data = []
    
    for row in row_list:
        x, y = extract_row_data(row)
        if y[0] > 0:                      # Данные добавляются, только если есть зарплата
            x_data.append(x)
            y_data.append(y)

    return np.array(x_data), np.array(y_data)

In [21]:
# Формирование выборки из загруженного набора данных    
x, y = construct_train_data(df.values)

In [22]:
# Форма наборов параметров и зарплат
print(x.shape) 
print(y.shape)

# Пример обработанных данных
n = 0 
print(x[n])
print(y[n])

(62967, 39)
(62967, 1)
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[40.]


### Преобразование текстовых данных

#### Подготовка текстовых данных "Должность"

In [23]:
# Функция извлечения данных о профессии

def extract_prof_text(row_list):
    result = []
    
    # Для всех строк таблицы: собрать значения 
    # столбцов желаемой и прошлой должности
    # если есть информация о зарплате
    
    for row in row_list:
        if extract_salary(row[COL_SALARY]) > 0:
            result.append(str(row[COL_POS_SEEK]) + ' ' + str(row[COL_POS_PREV]))
    
    # Возврат в виде массива
    return result

In [24]:
# Извлечение текстов о профессии для выборки
prof_text = extract_prof_text(df.values) 

# Пример текста о профессии из резюме
print(df.values[120])
print(prof_text[120]) 

['Мужчина ,  33 года , родился 8 октября 1985' '250000 руб.'
 'Руководитель BI' 'Москва , не готов к переезду , готов к командировкам'
 'полная занятость' 'полный день'
 'Опыт работы 11 лет 6 месяцев  Руководитель BI 250 000 руб. Информационные технологии, интернет, телеком Инженер Аналитик Занятость: полная занятость График работы: полный день Опыт работы 11 лет 6 месяцев Декабрь 2017 — по настоящее время 1 год 5 месяцев ЭркаФарм Россия , erkapharm.com Медицина, фармацевтика, аптеки ... Аптека, оптика Руководитель отдела аналитики Стратегическая цель: создание единой корпоративной автоматизированной информационно-аналитической системы бизнес анализа; MDM: ведение и поддержка в актуальном состоянии мастер-справочников компании; разработка новых справочников\\атрибутов для обеспечения деятельности подразделений компании; контроль бизнес-процессов в части, касающейся мастер-данных; написание документов, закрепляющих регламентные нормы по ведению мастер-справочников; ETL: загрузка данных 

In [25]:
# Преобразование текстовых данных в числовые/векторные для обучения нейросетью

# Используется встроенный в Keras токенизатор для разбиения текста и построения частотного словаря
prof_tokenizer = Tokenizer(num_words=3000,                                       # Объем словаря
                           filters='!"#$%&()*+,-–—./:;<=>?@[\\]^_`{|}~\t\n\xa0', # Убираемые из текста ненужные символы
                           lower=True,                                           # Приведение слов к нижнему регистру
                           split=' ',                                            # Разделитель слов
                           oov_token='unknown',                                  # Токен для слов, которые не вошли в словарь 
                           char_level=False)                                     # Указание разделять по словам, а не по единичным символам

prof_tokenizer.fit_on_texts(prof_text) # Обучаем токенайзер

In [26]:
# Преобразование текстов в последовательность индексов согласно частотному словарю
prof_seq = prof_tokenizer.texts_to_sequences(prof_text)

In [27]:
# Преобразование последовательностей индексов в bag of words
x_prof = prof_tokenizer.sequences_to_matrix(prof_seq)

In [28]:
# Проверка результата
print(x_prof.shape)

(62967, 3000)


#### Подготовка текстовых данных "Опыт работы"

In [29]:
# Функция извлечения данных описания опыта работы

def extract_exp_text(row_list):
    result = []
    
    # Для всех строк таблицы: собрать значения опыта работы,
    # если есть информация о зарплате
    for row in row_list:
        if extract_salary(row[COL_SALARY]) > 0:
            result.append(str(row[COL_EXP]))
    
    # Возврат в виде массива
    return result

In [30]:
# Извлечение текстов об опыте работы для выборки
exp_text = extract_exp_text(df.values) 

# Пример текста об опыте работы из резюме
print(df.values[120])
print(exp_text[120]) 

['Мужчина ,  33 года , родился 8 октября 1985' '250000 руб.'
 'Руководитель BI' 'Москва , не готов к переезду , готов к командировкам'
 'полная занятость' 'полный день'
 'Опыт работы 11 лет 6 месяцев  Руководитель BI 250 000 руб. Информационные технологии, интернет, телеком Инженер Аналитик Занятость: полная занятость График работы: полный день Опыт работы 11 лет 6 месяцев Декабрь 2017 — по настоящее время 1 год 5 месяцев ЭркаФарм Россия , erkapharm.com Медицина, фармацевтика, аптеки ... Аптека, оптика Руководитель отдела аналитики Стратегическая цель: создание единой корпоративной автоматизированной информационно-аналитической системы бизнес анализа; MDM: ведение и поддержка в актуальном состоянии мастер-справочников компании; разработка новых справочников\\атрибутов для обеспечения деятельности подразделений компании; контроль бизнес-процессов в части, касающейся мастер-данных; написание документов, закрепляющих регламентные нормы по ведению мастер-справочников; ETL: загрузка данных 

In [31]:
# Преобразование текстовых данных в числовые/векторные для обучения нейросетью

# Используется встроенный в Keras токенизатор для разбиения текста и построения частотного словаря
exp_tokenizer = Tokenizer(num_words=3000,                                       # Объем словаря
                          filters='!"#$%&()*+,-–—./:;<=>?@[\\]^_`{|}~\t\n\xa0', # Убираемые из текста ненужные символы
                          lower=True,                                           # Приведение слов к нижнему регистру
                          split=' ',                                            # Разделитель слов
                          oov_token='unknown',                                  # Токен для слов, которые не вошли в словарь 
                          char_level=False)                                     # Указание разделять по словам, а не по единичным символам

exp_tokenizer.fit_on_texts(exp_text) # Обучаем токенайзер

In [32]:
# Преобразование текстов в последовательность индексов согласно частотному словарю
exp_seq = exp_tokenizer.texts_to_sequences(exp_text)

In [33]:
# Преобразование последовательностей индексов в bag of words
x_exp = exp_tokenizer.sequences_to_matrix(exp_seq)

In [34]:
# Проверка результата
print(x_exp.shape)

(62967, 3000)


### Формирование выборок

In [35]:
x_train, x_test, x_train_prof, x_test_prof, x_train_exp, x_test_exp, y_train, y_test = train_test_split(x,
                                                                                                        x_prof,
                                                                                                        x_exp,
                                                                                                        y,
                                                                                                        test_size=0.1,
                                                                                                        random_state=42)

### Нормализация данных

In [36]:
# Для нормализации данных используется готовый инструмент
y_scaler = StandardScaler()

# Нормализация выходных данных обучающей выборки
y_train_scaled = y_scaler.fit_transform(y_train)

# Нормализация выходных данных тестовой выборки
y_test_scaled = y_scaler.transform(y_test)

In [37]:
# Проверка нормализации
print(y_train_scaled.shape)
print(f'Оригинальное значение зарплаты:  {y_train[1, 0]}')
print(f'Нормированное значение зарплаты: {y_train_scaled[1, 0]}')

# Вывод границ ненормализованных и нормализованных данных
print(y_train.mean(), y_train.std())
print(y_train_scaled.mean(), y_train_scaled.std())

(56670, 1)
Оригинальное значение зарплаты:  70.0
Нормированное значение зарплаты: -0.11018971877657593
77.97819040056469 72.40412707415571
-1.7829394216531895e-16 1.0


# Creat and fit model.
Здесь создайте и обучите модель нейронной сети с тремя входами для обработки:

In [38]:
input1 = Input((x_train.shape[1],))       # Вход нейронной сети для числовых данных
input2 = Input((x_train_prof.shape[1],))  # Вход нейронной сети для текстовых данных "должность"
input3 = Input((x_train_exp.shape[1],))   # Вход нейронной сети для текстовых данных "опыт работы"

x1 = input1                               # Ветка 1
x1 = Dense(128, activation="relu")(x1)
x1 = Dense(1000, activation="tanh")(x1)
x1 = Dense(100, activation="relu")(x1)

x2 = input2                               # Ветка 2
x2 = Dense(20, activation="relu")(x2)
x2 = Dense(500, activation="relu")(x2)
x2 = Dropout(0.3)(x2)

x3 = input3                               # Ветка 3
x3 = Dense(30, activation="relu")(x3)
x3 = Dense(800, activation="relu")(x3)
x3 = Dropout(0.3)(x3)

x = concatenate([x1, x2, x3])             # Объединение трех веток

x = Dense(15, activation='relu')(x)       # Промежуточный слой
x = Dropout(0.5)(x)
x = Dense(1, activation='linear')(x)      # Финальный регрессирующий нейрон

# Создаем модель нейронный сети при помощи класса Model
model = Model((input1, input2, input3), x)

2023-03-13 02:03:19.297214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 02:03:19.297437: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 02:03:19.297705: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-13 02:03:19.297917: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-13 02:03:19.297960: W tensorflow/c

In [39]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 39)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3000)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 3000)]       0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          5120        ['input_1[0][0]']                
                                                                                              

In [41]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-03-13 02:04:30.365394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 02:04:30.365545: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


False

In [42]:
# Компилируем модель
model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

# Обучаем модель
history = model.fit([x_train, x_train_prof, x_train_exp],
                    y_train_scaled,
                    batch_size=256,
                    epochs=50,
                    validation_data=([x_test, x_test_prof, x_test_exp], y_test_scaled),
                    shuffle=True,
                    verbose=1)

2023-03-13 02:04:37.363871: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 680040000 exceeds 10% of free system memory.


Epoch 1/50
222/222 [==============================] - 3s 10ms/step - loss: 0.1890 - mae: 0.2675 - val_loss: 0.3938 - val_mae: 0.3046
Epoch 2/50
222/222 [==============================] - 2s 9ms/step - loss: 0.1901 - mae: 0.2646 - val_loss: 0.3921 - val_mae: 0.2957
Epoch 3/50
222/222 [==============================] - 2s 9ms/step - loss: 0.1932 - mae: 0.2607 - val_loss: 0.3980 - val_mae: 0.2912
Epoch 4/50
222/222 [==============================] - 2s 9ms/step - loss: 0.1847 - mae: 0.2598 - val_loss: 0.3961 - val_mae: 0.2939
Epoch 5/50
222/222 [==============================] - 2s 10ms/step - loss: 0.1864 - mae: 0.2587 - val_loss: 0.3921 - val_mae: 0.2930
Epoch 6/50
222/222 [==============================] - 3s 11ms/step - loss: 0.1967 - mae: 0.2577 - val_loss: 0.3933 - val_mae: 0.2932
Epoch 7/50
222/222 [==============================] - 2s 11ms/step - loss: 0.1783 - mae: 0.2536 - val_loss: 0.3956 - val_mae: 0.2948
Epoch 8/50
222/222 [==============================] - 2s 11ms/step - los

# Задайте кастомную сигнатуру модели и сохраните обученную нейронную сеть. Выведите сигнатуру сохраненной модели.

In [44]:
# Создание кастомной сигнатуры с помощью класса tf.Module
class MyModule(tf.Module):
    def __init__(self, model):  # При инициализации экземпляра класса передаем модели нейронной сети
        self.model = model      # Присваиваем модель в переменную
    
    @tf.function(input_signature=([tf.TensorSpec(shape=(None, x_train.shape[1]), dtype=tf.float32)], 
                                  [tf.TensorSpec(shape=(None, x_train_prof.shape[1]), dtype=tf.float32)],
                                  [tf.TensorSpec(shape=(None, x_train_exp.shape[1]), dtype=tf.float32)])) # Вызываем декоратор, указывая формы тензоров, которые будут поступать на вход метода.
    def score(self, input_1, input_2, input_3):
        result = self.model([input_1, input_2, input_3]) # Подаем данные на вход модели
        return {"result": result}                      # Возвращаем результат
    
module = MyModule(model) # Вызываем класс MyModule

In [45]:
# Задаем сигнатуру. 
# PS: tо есть указываем, что в качестве входов и выходов модели, которую будем загружать на сервер,\ 
# должны использоваться входы и выходы метода score.

signatures = {"score": module.score}

In [48]:
# Save model
MODEL_DIR = './content/tmp/'                             # Путь к директории для хранения моделей
version = 1                                            # Номер версии модели
export_path = os.path.join(MODEL_DIR, str(version))    # Путь к директории для хранения модели с заданной версией


tf.saved_model.save(module,                # Передаем модель, которую нужно сохранить
                    export_path,           # Путь для сохранения
                    signatures=signatures) # Указываем сигнатуру модели

print('Путь для экспорта: {}'.format(export_path))


INFO:tensorflow:Assets written to: ./content/tmp/1/assets


INFO:tensorflow:Assets written to: ./content/tmp/1/assets


Путь для экспорта: ./content/tmp/1


In [49]:
!saved_model_cli show --dir {export_path} --all

2023-03-13 02:10:38.778557: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 02:10:38.914849: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-13 02:10:38.914867: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-13 02:10:39.792290: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Запуск tf-serving

## запуск на локальной машине. Исполнить команты в терминале

In [50]:
name_of_container = 'tf_serving_hh'
port = '8501:8501'
source = '/home/nikmih/Jupyter_projects/tensorflow_serving/content/tmp/'
target = '/models/hh_model/'
model_name = 'hh_model'
container_name = 'hh_container'

docker_run_comand = f'docker run -p {port} --name {container_name} --mount type=bind,source={source},target={target} \
-e MODEL_NAME={model_name} -t tensorflow/serving'
print(docker_run_comand)


'''
#containes list
docker sm
# docker stop container
docker stop 70f91474afac
# удалить контейнер
docker rm 
'''

docker run -p 8501:8501 --name hh_container --mount type=bind,source=/home/nikmih/Jupyter_projects/tensorflow_serving/content/tmp/,target=/models/hh_model/ -e MODEL_NAME=hh_model -t tensorflow/serving


'\n#containes list\ndocker sm\n# docker stop container\ndocker stop 70f91474afac\n# удалить контейнер\ndocker rm \n'

# Проверка вывода запроса

## Отправим POST запрос через requests

In [52]:
import json # Модуль для кодирования и декодирования данных в формате json

# Передаем данные для записи в формат JSON
data = json.dumps({"signature_name": "score", "inputs": {"input_1": x_test[:10].tolist(),
                                                         "input_2": x_test_prof[:10].tolist(),
                                                         "input_3": x_test_exp[:10].tolist()}}) 

In [56]:
import requests # Модуль для составления HTTP-запросов

# Указываем, что будем отправлять данные в формате JSON
headers = {"content-type": "application/json"} 

# Отправляем данные на сервер и получаем результат
json_response = requests.post('http://localhost:8501/v1/models/hh_model:predict', data=data, headers=headers)

In [58]:
# Записываем предсказания в переменную
# prediction = json.loads(json_response.text)
prediction = json.loads(json_response.text)['outputs']

prediction

[[-0.700656176],
 [-0.515386581],
 [-0.724390447],
 [-0.251978427],
 [-0.239542037],
 [-0.583382249],
 [-0.0531558394],
 [1.10891211],
 [-0.09735322],
 [0.157538652]]

In [59]:
# Возвращаем данные к исходному диапазону значений
unscaled_pred = y_scaler.inverse_transform(prediction)

## Сравним с реальными примерами

In [60]:
for i in range(len(unscaled_pred)):
    print('Реальное значение: {:6.2f}  Предсказанное значение: {:6.2f}  Разница: {:6.2f}'.format(y_test[i, 0],
                                                                                                 unscaled_pred[i, 0],
                                                                                                 abs(y_test[i, 0] - unscaled_pred[i, 0])))

Реальное значение:  29.00  Предсказанное значение:  27.25  Разница:   1.75
Реальное значение:  40.00  Предсказанное значение:  40.66  Разница:   0.66
Реальное значение:  20.00  Предсказанное значение:  25.53  Разница:   5.53
Реальное значение: 100.00  Предсказанное значение:  59.73  Разница:  40.27
Реальное значение: 140.00  Предсказанное значение:  60.63  Разница:  79.37
Реальное значение:  25.00  Предсказанное значение:  35.74  Разница:  10.74
Реальное значение:  80.00  Предсказанное значение:  74.13  Разница:   5.87
Реальное значение: 250.00  Предсказанное значение: 158.27  Разница:  91.73
Реальное значение:  70.00  Предсказанное значение:  70.93  Разница:   0.93
Реальное значение:  65.00  Предсказанное значение:  89.38  Разница:  24.38
